# Propagation using Kirchhof-Helmholtz diffraction integral

Propagation used to reconstruct holograms obtained by using a point source from which a spherical wave comes out with wavelength $\lambda$. The implementation allows any regular sampling of the output window at a plane for constant $z$.

## From [Kreuzer patent](https://patents.google.com/patent/US6411406B1/en)

The diffraction integral is
$$K(\overrightarrow{r}) = \int_S d^2\xi I(\xi) e^{ik\frac{\overrightarrow{\xi}\cdot\overrightarrow{r}}{\xi}},$$
where the wave number is $k=2\pi/\lambda$, $i=\sqrt{-1}$, $S$ is the surface where the hologram $I(\xi)$ (2D intensity image) is, $\overrightarrow{\xi} = (X,Y,L)$ are the coordinates in the hologram that is located $L$ away from the point source, and $\overrightarrow{r} = (x,y,z)$ is the distance from the point source to the points in the output window.

The following coordinates transform is applied:

$$X' = \frac{XL}{R}, \; Y' = \frac{XL}{R}, \; R = \sqrt{L^2+X^2+Y^2}.$$

That brings:

$$X = \frac{X'L}{R'}, \; Y = \frac{X'L}{R'}, \; R' = \sqrt{L^2-X'^2-Y'^2}.$$

The hologram is sampled (interpolated) into a regular grid on $(X',Y',L)$:

$$X'_p = X'_0 + p \Delta X', \; Y'_q = Y'_0 + q \Delta Y', \; p=0,\dots, P-1, \; q=0,\dots, Q-1.$$

And the output window is sampled using $x_n = x_0 + n\delta_x, \; y_m = y_0 + m\delta_y, \; n=0,\dots, N-1, \; m=0,\dots, M-1.$ The diffraction integral becomes a 2D convolution (eq. 6 in patent) when we use the identity $np=[n^2+p^2—(n—p)^2]/2$ and $mq = [m^2+q^2—(m—q)^2]/2$:

$$K_{nm} = \Delta X' \Delta Y' e^{\frac{ik}{L}(x_nX'_0 + y_mY'_0)} e^{\frac{ik}{2L} (n^2\delta_x\Delta X' + m^2\delta_y\Delta Y')} \mbox{IFFT}\{\tilde{K}'_{uv} \tilde{R}_u \tilde{R}_v\},$$

$$\tilde{K}'_{uv} = \mbox{FFT}\{K'_{pq}\} = \mbox{FFT}\left\{I'_{pq} e^{\frac{ik}{L}(px_0\Delta X' + qy_0 \Delta Y' + \frac{p^2 \delta_x\Delta X'}{2} + \frac{q^2 \delta_y\Delta Y'}{2})}\right\}$$

$$I'_{pq} = I(X(X'_p,Y'_q),Y(X'_p,Y'_q)) \left(\frac{L}{R'}\right)^4 e^{ikz\frac{R'}{L}}$$

$$\tilde{R}_u = \mbox{FFT}\left\{R_s\right\} = \mbox{FFT}\left\{ e^{-\frac{ik}{2L}s^2\delta_x\Delta X'} \right\}, \; \tilde{R}_v = \mbox{FFT}\left\{ R_t \right\} = \mbox{FFT}\left\{ e^{-\frac{ik}{2L}t^2\delta_y\Delta Y'} \right\}$$

## Sampling issues

A proper sampling of $K'_{pq}$ is obtained if the phase factor in the exp term $\Phi_K = kz\frac{R'}{L} + \frac{k}{L}(px_0\Delta X' + qy_0 \Delta Y' + \frac{p^2 \delta_x\Delta X'}{2} + \frac{q^2 \delta_y\Delta Y'}{2})$, has $\left|\frac{\partial \Phi_K}{\partial p}\right| \leq \pi$, or, for fixed $\Delta X'$ and $P$ (proper sampling of the hologram), and solving for $q=0$ as it produces the highest gradients in $p$:

$$\delta_x \leq \frac{1}{P}\left(\frac{L \lambda}{2 \Delta X'} - x_0 + \frac{z}{R'}X'_{P-1}\right)\mbox{, similar for }\delta_y.$$

This condition is usually met.

The other terms $R_s$ and $R_t$ need to be properly sampled too, and they usually aren't. As these terms are convolved with $K'_{pq}$, they are evaluated as e.g. $R_{n-p}$, then $s = -P+1, \dots, N-1$ and $t = -Q+1, \dots, M-1$. So, $R_s$ has to be properly sampled up to $s= \hat{S} = max(P-1, N-1)$ and $R_t$ up to $t = \hat{T} = \max(Q-1,M-1)$.

As the contributions from a relatively small particle in $I$ take part mainly around them, it is probably a good option to truncate $R_s$ and $R_t$ where they begin to be incorrectly sampled, giving $\hat{S}' \leq \hat{S}$ and $\hat{T}' \leq \hat{T}$. Otherwise, $\delta_x$ and $\delta_y$ could be changed to have e.g. $\left|\frac{\partial \Phi_R}{\partial s}\right| \leq \pi$:

$$\delta_x \leq \frac{\lambda L}{2\hat{S}\Delta X'}, \; \delta_y \leq \frac{\lambda L}{2\hat{T}\Delta Y'}$$

But if the same output window is expected, then $N$ and $M$ should change to $N' = N\frac{\delta_x^{ini}}{\delta_x^{new}}$ and $M' = M\frac{\delta_y^{ini}}{\delta_y^{new}}$, which, for $N' > P, (\hat{S} = N')$ reaches the condition:

$$2 N \Delta X' \delta_x^{ini} \leq \lambda L.$$

## Border effects

Usually, the Kirchhoff-Helmholtz diffraction integral (KHDI) is used having a contrast hologram as input instead of the direct hologram. The contrast hologram is calculated from a hologram $I^H$ and a background image $I^B$ as $I = \frac{I^H - I^B}{I^B}$. The backgroun image is the intensity that the camera senses without particles in the sampled volume. The hologram has a mean value in most pixels and diffraction patterns coming from a particle show as circular fringes. This way, $I$ is usually 0 in the borders, so it's okay to use zero padding when calculating the convolution for the KHDI.

In our case, we'll use $I = I^H/I^B$ so that $I\equiv 1$ close to the borders. In this case, is preferable to pad with $(\hat{S}', \hat{T}')$ 1s before performing the convolution.